# Surgical Site Infection (SSI)

## Related Factors for predicting SSI
- Patient
  - age - mimiciv_hosp.patients, mimiciv_derived.icustay_detail (admission age)
  - gender - mimiciv_hosp.patients, mimiciv_derived.icustay_detail
  - race - mimiciv_derived.icustay_detail
  - (?) height - mimiciv_derived.height
- Treatments
  - Antibiotic usage
- Patient Condition
  - chart events
    - changes in temperature, current temperature
    - skin condition
    

# Setup


In [123]:
#### From Tutorial03.ipynb
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
# Authenticate
auth.authenticate_user()

# Set up environment variables
project_id = 'ml-health-application'   ### Please change project_id
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

# Overview
- ICDs
- Patients (overall about patients with SSI ICDS)
  - 4295 patients flagged under above icds
  - 1185 patients flagged under above icds and stay in icu
  - SSI that continue to be sepsis 758 patients
  - 963 patients suspected having infection

## ICD codes


In [ ]:
icd_query = """SELECT *
FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
WHERE (lower(long_title) LIKE '%surgical%' AND lower(long_title) LIKE '%infection%') or icd_code like "9985%" """
table = run_query(icd_query)
ssi_icds = list(table["icd_code"])
print("--- icd_codes list ---")
print(ssi_icds)
print("---table---")
table

--- icd_codes list ---
['99851', '99859', 'O860', 'O8600', 'O8601', 'O8602', 'O8603', 'O8609', 'T8141', 'T8141XA', 'T8141XD', 'T8141XS', 'T8142', 'T8142XA', 'T8142XD', 'T8142XS', 'T8143', 'T8143XA', 'T8143XD', 'T8143XS', 'T8149', 'T8149XA', 'T8149XD', 'T8149XS']
---table---


,icd_code,icd_version,long_title
0,99851,9,Infected postoperative seroma
1,99859,9,Other postoperative infection
2,O860,10,Infection of obstetric surgical wound
3,O8600,10,"Infection of obstetric surgical wound, unspeci..."
4,O8601,10,"Infection of obstetric surgical wound, superfi..."
5,O8602,10,"Infection of obstetric surgical wound, deep in..."
6,O8603,10,"Infection of obstetric surgical wound, organ a..."
7,O8609,10,"Infection of obstetric surgical wound, other s..."
8,T8141,10,"Infection following a procedure, superficial i..."
9,T8141XA,10,"Infection following a procedure, superficial i..."


## Patients

### Patients with following icds

In [ ]:
# (4295 patients flagged under above icds)
patient_query = f"""select subject_id , hadm_id FROM `physionet-data.mimiciv_hosp.diagnoses_icd` where icd_code in ({str(ssi_icds)[1:-1]})
group by subject_id, hadm_id"""
run_query(patient_query)

,subject_id,hadm_id
0,17311271,23348535
1,16120907,23202662
2,16285950,22978966
3,10005858,29352282
4,10009657,23182574
...,...,...
4290,19206592,21154874
4291,10679181,21982353
4292,14729643,21468281
4293,10413295,27286678


### Patients with ICU stay

In [ ]:
# 1185 patients flagged under above icds and stay in icu

icu_stay_query = f"""SELECT a.* FROM `physionet-data.mimiciv_icu.icustays` as a
  join (
    {patient_query}
  ) as b on a.subject_id = b.subject_id and a.hadm_id = b.hadm_id"""
run_query(icu_stay_query)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,12044903,27228124,37187123,Neuro Intermediate,Neuro Intermediate,2188-04-20 22:50:00,2188-04-24 23:05:13,4.010567
1,12242977,26428875,36962214,Neuro Intermediate,Neuro Intermediate,2154-03-27 15:14:41,2154-04-04 15:31:42,8.011817
2,12242977,26428875,38003809,Neuro Intermediate,Neuro Intermediate,2154-03-08 20:35:00,2154-03-24 16:48:59,15.843044
3,12972983,26591054,34159900,Neuro Intermediate,Neuro Intermediate,2173-12-20 19:41:00,2173-12-28 15:07:59,7.810405
4,13825646,23969955,35411450,Neuro Intermediate,Neuro Intermediate,2121-05-21 21:05:52,2121-05-23 19:31:21,1.934363
...,...,...,...,...,...,...,...,...
1180,19689858,22703972,32317319,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2146-06-23 15:05:02,2146-06-24 20:11:18,1.212685
1181,19777866,29857973,38405028,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2151-06-07 00:27:53,2151-06-08 20:41:53,1.843056
1182,19813160,25726255,34573170,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2176-10-03 20:56:00,2176-10-04 10:17:51,0.556840
1183,19813160,25726255,35268224,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2176-10-04 10:24:57,2176-10-04 19:35:08,0.382072


### Following patients with sepsis

In [ ]:
# surgical site infection that continue to be sepsis 758 patients
query = f"""SELECT c.* FROM `physionet-data.mimiciv_derived.sepsis3` as c
join (
    {icu_stay_query}
) as d on c.subject_id = d.subject_id and c.stay_id = d.stay_id"""
run_query(query)

,subject_id,stay_id,antibiotic_time,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,13825646,35411450,2121-05-23 10:00:00,2121-05-21 23:25:00,2121-05-21 23:25:00,2121-05-21 23:00:00,2,0,0,0,1,1,0,True
1,16302564,35595370,2115-07-01 13:00:00,2115-06-29 11:36:00,2115-06-29 11:36:00,2115-06-30 01:00:00,4,0,0,0,1,0,3,True
2,16302564,37533860,2115-07-21 18:00:00,2115-07-20 21:13:00,2115-07-20 21:13:00,2115-07-18 22:00:00,2,0,0,0,1,1,0,True
3,10193237,31311091,2112-02-07 08:00:00,2112-02-06 20:00:00,2112-02-06 20:00:00,2112-02-07 02:00:00,2,0,0,0,1,0,1,True
4,10300586,37500831,2140-04-14 20:00:00,2140-04-12 11:15:00,2140-04-12 11:15:00,2140-04-13 03:00:00,2,0,0,0,1,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,19125782,35677228,2176-10-02 10:00:00,2176-09-30 19:09:00,2176-09-30 19:09:00,2176-09-30 23:00:00,4,0,1,0,1,0,2,True
754,19351106,30187303,2130-08-08 06:00:00,2130-08-07 07:45:00,2130-08-07 07:45:00,2130-08-06 21:00:00,3,0,0,0,0,3,0,True
755,19565716,30887451,2187-07-26 21:00:00,2187-07-25 23:01:00,2187-07-25 23:01:00,2187-07-26 17:00:00,5,0,1,4,0,0,0,True
756,19593791,37904166,2145-01-25 21:00:00,2145-01-25 19:22:00,2145-01-25 19:22:00,2145-01-25 22:00:00,2,0,0,0,1,0,1,True


### Suspected patients with infection



In [ ]:
# 963 patients suspected having infection
query = f"""SELECT distinct c.subject_id, c.stay_id FROM `physionet-data.mimiciv_derived.suspicion_of_infection` as c
join (
  {icu_stay_query}
) as d on c.subject_id = d.subject_id and c.stay_id = d.stay_id and suspected_infection = 1"""
run_query(query)

,subject_id,stay_id
0,12044903,37187123
1,12242977,36962214
2,13825646,35411450
3,13925251,36138065
4,15317213,37037726
...,...,...
958,19593791,37904166
959,19682346,30070110
960,19689280,35827337
961,19813160,34573170


## Treatment

### Antibiotics usage


In [ ]:
# antibiotic used for patients with these ICDs (84 records)
antibiotic_query = f"""SELECT distinct a.antibiotic FROM `physionet-data.mimiciv_derived.antibiotic` as a
join (
  {patient_query}
) as b on a.subject_id = b.subject_id and a.hadm_id = b.hadm_id"""
run_query(antibiotic_query)

                                antibiotic
0                                   doxycy
1                                 Amikacin
2                                 CefePIME
3                                 RifAMPin
4                                 Rifampin
..                                     ...
79     Ampicillin-Sulbact Graded Challenge
80  Erythromycin Ethylsuccinate Suspension
81              Tobramycin Inhalation Soln
82                  AMOXicillin Oral Susp.
83             Gentamicin Intraventricular

[84 rows x 1 columns]


## Patient Condition

### Chart events
- Temperature, Changes in Temperature
- Skin condition (including odor) from  (category = "Skin - Impairment")

In [ ]:
skin_con_query = f"""
SELECT a.subject_id, a.stay_id, a.hadm_id, c.abbreviation, a.value, a.charttime FROM `physionet-data.mimiciv_icu.chartevents` as a
join (
  {patient_query} limit 100
) as b on a.subject_id = b.subject_id and a.hadm_id = b.hadm_id
join(
  SELECT * FROM `physionet-data.mimiciv_icu.d_items`
) as c on a.itemid = c.itemid and category = "Skin - Impairment"
"""
run_query(skin_con_query)

,subject_id,stay_id,hadm_id,abbreviation,value,charttime
0,16120907,37523171,23202662,Site/Wound Base #1,Pink,2198-04-06 08:46:00
1,16120907,37523171,23202662,Surrounding Tissue #2,Intact,2198-03-18 04:00:00
2,16120907,37523171,23202662,Impaired Skin #1- Type,Blister,2198-04-05 12:00:00
3,16120907,37523171,23202662,Surrounding Tissue #2,Intact,2198-03-26 01:00:00
4,16120907,37523171,23202662,Site/Wound Base #3,Pink,2198-03-20 20:00:00
...,...,...,...,...,...,...
4914,16120907,37523171,23202662,Odor #3,0,2198-04-01 03:43:00
4915,16120907,37523171,23202662,Odor #2,0,2198-04-05 05:27:00
4916,16120907,37523171,23202662,Site Length #2,5,2198-04-01 16:00:00
4917,16120907,37523171,23202662,Site Width #1,6,2198-03-16 16:25:00


### Surgery
Definition of curr_service
https://github.com/MIT-LCP/mimic-iv-website/blob/master/content/hosp/services.md

In [128]:
query = """SELECT distinct curr_service FROM `physionet-data.mimiciv_hosp.services` where curr_service like "%SURG" LIMIT 1000 """

run_query(query)

,curr_service
0,SURG
1,CSURG
2,NSURG
3,PSURG
4,TSURG
5,VSURG
